In [1]:
!pip install transformers torch


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [12]:
pip install wikipedia


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=bde8761d33633fd0571197dc2669c804dd99ed77de767e02cd1c0cf73c974a75
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [13]:
import torch
from transformers import BertForQuestionAnswering, BertTokenizer
import wikipedia

# Load pre-trained model and tokenizer
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

def get_wikipedia_context(query):
    wikipedia.set_lang("en")
    try:
        # Fetch a summary of the page
        summary = wikipedia.summary(query, sentences=3)  # Limit to 3 sentences for brevity
        return summary
    except wikipedia.exceptions.DisambiguationError as e:
        return "Your query is ambiguous. Try being more specific."
    except wikipedia.exceptions.PageError as e:
        return "Sorry, I couldn't find any information on that topic."

def generate_response(question):
    # Fetch context from Wikipedia
    context = get_wikipedia_context(question)

    # Tokenize input
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")

    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    # Get the model's answer
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    answer_start_scores, answer_end_scores = outputs.start_logits, outputs.end_logits
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    # Convert tokens to string
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end]))

    return answer

def chatbot():
    print("Hi, I'm a chatbot created to assist you. Type 'quit' to exit")

    while True:
        # Get user input
        user_input = input("You: ")

        if user_input.lower() == "quit":
            print("Chatbot: Bye for now. See you soon :)")
            break

        # Generate response
        response = generate_response(user_input)
        print(f"Chatbot: {response}")

if __name__ == "__main__":
    chatbot()


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Hi, I'm a chatbot created to assist you. Type 'quit' to exit
You: What is the GDP of India
Chatbot: gross domestic product
You: Can you explain brownian motion?
Chatbot: the random motion of particles suspended in a medium
You: where is Kolkata?
Chatbot: on the eastern bank of the hooghly river
You: in which part of India is Mumbai situated?
Chatbot: maharashtra
You: Where is the location of Victoria Memorial?
Chatbot: the maidan in central kolkata
You: where is India Gate located?
Chatbot: near the kartavya path on the eastern edge of the " ceremonial axis " of new delhi
You: where is Taj Hotel?
Chatbot: mumbai , india
You: Which is the national language of India?
Chatbot: indo - aryan
You: what is the full form of MICR in banking?
Chatbot: micr
You: give the full form of "BERT" in deep learning
Chatbot: haldane ' s decompression model is a mathematical model for decompression to sea level atmospheric pressure of divers breathing compressed air at ambient pressure that was proposed in